# Spam Filter with the Naive Bayes Algorithm
In this project, we build a spam filter for SMS messages.
In order to classify messages as spam or non-spam, the high-level algorithm:
1. learns how humans classify messages;
2. uses that human knowledge to estimate probabilities that new messages are either spam or non-spam;
3. classifies a new message using these probability values.

We will use a **multinomial Naive Bayes algorithm** and work with a dataset of 5'572 SMS messages that have already been classified by humans. This dataset can be downloaded from the [UCI ML Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection).

## 1. Exploring the Dataset

In [5]:
import pandas as pd
import numpy as np

Read `SMSSpamCollection` into a Pandas DataFrame using the tab delimiter (`\t`). Since the dataset does not have a header row, we use the `header=None` option. Moreover, we use the `names=['Label', 'SMS']` parameter to name the columns as Label and SMS.

In [6]:
# MS: Read the data into a Pandas DataFrame
df = pd.read_csv("SMSSpamCollection", sep = "\t", header = None,
                 names=['Label', 'SMS'])

df.head() # print the first 5 rows

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [7]:
df.shape

(5572, 2)

The dataset has 5572 rows (as announced above) and 2 columns.

Let's find out the percentages of spam and non-spam (aka "ham") messages.

In [8]:
percent_spam = (df[df['Label']=='spam'].shape[0]/df.shape[0])*100
percent_ham = (df[df['Label']=='ham'].shape[0]/df.shape[0])*100
print('% of spam messages:', round(percent_spam, 2))
print('% of ham messages: ', round(percent_ham, 2))

% of spam messages: 13.41
% of ham messages:  86.59


## 2. Training and Test Set
Before creating the spam filter, let's think and design the test. To test the spam filter, we split the dataset into two categories:
- a **training set**, which we use to train the algorithm how to classify messages;
- a **test set**, which we use to test how good the spam filter is in classifying new messages.

#### 1. Randomize the entire dataset
Before splitting the dataset, we randomize the entire dataset to ensure that spam and ham messages are spread properly throughout the dataset. The parameter `frac=1` ensures that the entire dataset is randomized, while `random_state=1` makes sure that the results are reproducible.

In [9]:
df = df.sample(frac=1, random_state=1)
df.head()

,Label,SMS
1078,ham,"Yep, by the pretty sculpture"
4028,ham,"Yes, princess. Are you going to make me moan?"
958,ham,Welp apparently he retired
4642,ham,Havent.
4674,ham,I forgot 2 ask ü all smth.. There's a card on ...


#### 2. Split the randomized dataset into a training and a test set
We choose a 80% - 20% split, in order to train the algorithm on as much data as possible, but still having enough data for testing.

In [10]:
nb_rows = df.shape[0]
nb_rows_train_set = round(nb_rows * .80)
train_df = df.iloc[0:nb_rows_train_set].copy()
test_df = df.iloc[nb_rows_train_set:].copy()

In [11]:
train_percent_spam = (train_df[train_df['Label']=='spam'].shape[0]/train_df.shape[0])*100
train_percent_ham = (train_df[train_df['Label']=='ham'].shape[0]/train_df.shape[0])*100
print('% of spam messages in the training set:', round(train_percent_spam, 2))
print('% of ham messages in the training set: ', round(train_percent_ham, 2))
test_percent_spam = (test_df[test_df['Label']=='spam'].shape[0]/test_df.shape[0])*100
test_percent_ham = (test_df[test_df['Label']=='ham'].shape[0]/test_df.shape[0])*100
print('% of spam messages in the test set:', round(test_percent_spam, 2))
print('% of ham messages in the test set: ', round(test_percent_ham, 2))

% of spam messages in the training set: 13.46
% of ham messages in the training set:  86.54
% of spam messages in the test set: 13.2
% of ham messages in the test set:  86.8


The percentages are similar to those we calculated for the full dataset.

## 3. Letter Case and Punctuaction
$ \displaystyle P(Spam\mid w_1, w_2, \ldots, w_n) \propto P(Spam) \cdot \prod_{i=1}^{n} P(w_{i}\mid Spam), $

where we compute the conditional probabilities with an additive smoothing:

$ \displaystyle P(w_{i} \mid Spam) = \frac{N_{w_{i}\mid Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{\text{vocabulary}}} $

Here, $ N_{w_{i}\mid Spam} $ is the number of times the word $ w_{i} $ occurs in spam messages. $ N_{Spam} $ is the total number of words in spam messages. $ N_{\text{vocabulary}} $ is the total number of words in the vocabulary. $ \alpha = 1 $ is the smoothing parameter.

We first need to perform some transformation of the data. We replace the `SMS` column by a series of new columns, where each column corresponds to a unique word from the vocabulary.

- All words in the vocabulary are in lower case.
- Punctuation is not taken into account.

In the following, for each message, we remove all the punctuation, transform every letter in every word to lower case, and split the string at the space character.

In [12]:
import re

def clean_message(message):
    # MS: We use the regex '\W' to detect any character that is not
    #     from a-z, A-Z, or 0-9.
    message = re.sub('\W', ' ', message)
    # MS: For each message, we transform every letter in every word
    #     to lower case, and split the string at the space character.
    message = message.lower().split()
    # print(message)
    return(message)

In [13]:
train_df['SMS'] = train_df['SMS'].apply(clean_message)

## 4. Creating the Vocabulary
Let's now create a **vocabulary**, i.e., a list with all of the _unique_ words that occur in the messages of our training set.

In [14]:
vocabulary = []
for sms in train_df['SMS']:
    # MS: For each message in the 'SMS' column...
    for word in sms:
        # ... we append each word to the vocabulary list:
        vocabulary.append(word)

# MS: We transform the vocabulary list into a set.
#     This removes the duplicates from the vocabulary list.
#     Then we transorm the vocabulary set back into a list.
vocabulary = list(set(vocabulary))

# MS: Print the first 10 words in the vocabulary.
print(vocabulary[0:10])

['15541', 'ymca', 'parade', 'gail', 'remain', 'poboxox36504w45wq', 'customersqueries', 'greece', 'may', '14tcr']


## 5. The Final Training Set
We will bring our training set to a format where each column represents a unique word in our vocabulary, showing the frequency of that word in a given message.

We first build a **dictionary** that we will then convert to a Pandas DataFrame. This dictionary will have as _keys_ the unique words from the vocabulary we created above, and for each key it will have as a _value_ a list of length of the training, where each element represents the number of times the key (i.e., the word) appears in the current message.

In [15]:
word_counts_per_sms = {unique_word: [0]*nb_rows_train_set for unique_word in vocabulary}

for index, sms in enumerate(train_df['SMS']):
    # MS: 'index' is the index of the sms in the dataframe train_df,
    #     and it is used to access the entries of the list related
    #     to each word in the dictionary 'word_counts_per_sms'.
    for word in sms:
        # For the message corresponding to 'index',
        # increment the counter for the current word by 1.
        word_counts_per_sms[word][index] += 1

We transform word_counts_per_sms into Pandas DataFrame and concatenate it with the train_df DataFrame we had before.

In [16]:
word_counts_per_sms_df = pd.DataFrame(word_counts_per_sms)

In [17]:
word_counts_per_sms_df.iloc[0:10, 3000:3010]

,french,frens,frequently,fresh,freshers,fret,fri,friday,fridge,friend
0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0


In [18]:
train_df_clean = pd.concat([train_df, word_counts_per_sms_df], axis=1)

Let's visualize a portion of the dataframe so created.

In [19]:
train_df_clean.iloc[0:5, 0:5]

,Label,SMS,0,00,000
0,ham,"[go, until, jurong, point, crazy, available, o...",0.0,0.0,0.0
1,ham,"[ok, lar, joking, wif, u, oni]",0.0,0.0,0.0
2,NaN,NaN,0.0,0.0,0.0
3,ham,"[u, dun, say, so, early, hor, u, c, already, t...",0.0,0.0,0.0
4,ham,"[nah, i, don, t, think, he, goes, to, usf, he,...",0.0,0.0,0.0


## 6. Calculating Constants
To be able to classify new messages, the Naive Bayes algorithm needs to know the values of the probabilities appearing in the following equations:
$ \displaystyle P(Spam\mid w_1, w_2, \ldots, w_n) \propto P(Spam) \cdot \prod_{i=1}^{n} P(w_{i}\mid Spam), $

$ \displaystyle P(Ham\mid w_1, w_2, \ldots, w_n) \propto P(Ham) \cdot \prod_{i=1}^{n} P(w_{i}\mid Ham), $

where

$ \displaystyle P(w_{i} \mid Spam) = \frac{N_{w_{i}\mid Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{\text{vocabulary}}}, $

$ \displaystyle P(w_{i} \mid Ham) = \frac{N_{w_{i}\mid Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{\text{vocabulary}}}. $

$ N_{Spam} $ is equal to the number of words in all the spam messages — it's not equal to the number of spam messages, and it's not equal to the total number of unique words in spam messages. Analogously for $ N_{Ham} $. We will use Laplace smoothing and set $ \alpha = 1 $.

In [20]:
spam_messages = train_df_clean[train_df_clean['Label']=='spam']
ham_messages = train_df_clean[train_df_clean['Label']=='ham']

# MS: We calculate P(Spam) and P(Ham).
p_spam = len(spam_messages)/len(train_df_clean)
p_ham = len(ham_messages)/len(train_df_clean)

# N_Spam and N_Ham.
# We apply the function 'len' to each row of the 'SMS' column
# of the spam_messages dataframe to compute the number of words
# in each message, then we use np.sum() to get the total number
# of words in all the spam messages.
n_words_per_spam_message = spam_messages['SMS'].apply(len)
n_spam = np.sum(n_words_per_spam_message)
print(n_spam)

# Analogously for the ham_messages dataframe.
n_words_per_ham_message = ham_messages['SMS'].apply(len)
n_ham = np.sum(n_words_per_ham_message)
print(n_ham)

# N_Vocabulary
n_vocabulary = len(vocabulary)
print(n_vocabulary)

# Laplace smoothing
alpha = 1

15190
57237
7783


## 7. Calculating Parameters
All the values computed in the previous section are constant values, in the sense that they do not depend on each message or each individual word in a message. On the contrary, $ P(w_{i}\mid Spam) $ and $ P(w_{i}\mid Ham) $ vary depending on individual words:

$ \displaystyle P(w_{i} \mid Spam) = \frac{N_{w_{i}\mid Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{\text{vocabulary}}}, $

$ \displaystyle P(w_{i} \mid Ham) = \frac{N_{w_{i}\mid Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{\text{vocabulary}}}. $

We emphasize that $ P(w_{i}\mid Spam) $ only depends on the training set, and as long as we do not change the training set, $ P(w_{i}\mid Spam) $ stays constant. _This means that we can use our training set to calculate the probability for each word in our vocabulary._

We have 7'783 words in our vocabulary, so we have to compute a total of 15'566 probabilities, since for each word we have to compute both $ P(w_{i}\mid Spam) $ and $ P(w_{i}\mid Ham) $.

_Rephrase all of this part: In more technical language, the probability values that P(wi|Spam) and P(wi|Ham) will take are called parameters.
The fact that we calculate so many values before even beginning the classification of new messages makes the Naive Bayes algorithm very fast (especially compared to other algorithms). When a new message comes in, most of the needed computations are already done, which enables the algorithm to almost instantly classify the new message.
If we didn't calculate all these values beforehand, then all these calculations would need to be done every time a new message comes in. _

We recall that $ P(w_{i}\mid Spam) $ and $ P(w_{i}\mid Ham) $ are key parts of the equations that we will use to classify new messages:

$ \displaystyle P(Spam\mid w_1, w_2, \ldots, w_n) \propto P(Spam) \cdot \prod_{i=1}^{n} P(w_{i}\mid Spam), $

$ \displaystyle P(Ham\mid w_1, w_2, \ldots, w_n) \propto P(Ham) \cdot \prod_{i=1}^{n} P(w_{i}\mid Ham). $

In [21]:
# We initialize two dictionaries, in which we will store the parameters
# for P(wi|Spam) and P(wi|Ham), respectively.
dictionary_p_wi_spam = dict( zip(vocabulary, [0]*len(vocabulary)) )
dictionary_p_wi_ham = dict( zip(vocabulary, [0]*len(vocabulary)) )

# We isolate the spam and the ham messages in the training set into
# two different DataFrames.
spam_train_df = train_df_clean[train_df_clean['Label']=='spam']
ham_train_df = train_df_clean[train_df_clean['Label']=='ham']

# Now we iterate over the vocabulary and for each word we calculate
# P(wi|Spam) and P(wi|Ham).

for wi in vocabulary:
    n_wi_spam = np.sum(spam_train_df[wi])
    dictionary_p_wi_spam[wi] = (n_wi_spam + alpha) / (n_spam + alpha * n_vocabulary)
    
    n_wi_ham = np.sum(ham_train_df[wi])
    dictionary_p_wi_ham[wi] = (n_wi_ham + alpha) / (n_ham + alpha * n_vocabulary)

## 8. Classifying a New Message
Now that we've calculated all the constants and parameters we need, we can start creating the spam filter. 

$ \displaystyle P(Spam\mid w_1, w_2, \ldots, w_n) \propto P(Spam) \cdot \prod_{i=1}^{n} P(w_{i}\mid Spam), $

$ \displaystyle P(Ham\mid w_1, w_2, \ldots, w_n) \propto P(Ham) \cdot \prod_{i=1}^{n} P(w_{i}\mid Ham). $

In [26]:
def classify(message):

    message = re.sub('\W', ' ', message).lower().split()

    # Initializations
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    # Now we perform the calculationds corresponding to the
    # formulas above.
    for word in message:
        if word in dictionary_p_wi_spam:
            p_spam_given_message *= dictionary_p_wi_spam[word]
        if word in dictionary_p_wi_ham:
            p_ham_given_message *= dictionary_p_wi_ham[word]
            
        # If the word is not present in any of the two dictionaries,
        # then don't do anything. 
        # We ignore words that are not part of the vocabulary.

    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

_Use the classify() function to classify two new messages. You can use any messages you want, but we suggest that one message is obviously spam, and the other is obviously ham. For instance, you can use these two messages:_

In [23]:
sms1 = 'WINNER!! This is the secret code to unlock the money: C3421.'
sms2 = "Sounds good, Tom, then see u there"

In [27]:
classify(sms1)

P(Spam|message): 4.1578268320785084e-29
P(Ham|message): 1.284446514392249e-25


'ham'

In [25]:
classify(sms2)

P(Spam|message): 2.4449942502228983e-24
P(Ham|message): 4.287484325865619e-22
Label: Ham


## 9. Measuring the Spam Filter's Accuracy

In [54]:
def classify_test_set(message):

    message = re.sub('\W', ' ', message).lower().split()
    
    # Initializations
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    # Now we perform the calculationds corresponding to the
    # formulas above.
    for word in message:
        if word in dictionary_p_wi_spam:
            p_spam_given_message *= dictionary_p_wi_spam[word]
        if word in dictionary_p_wi_ham:
            p_ham_given_message *= dictionary_p_wi_ham[word]
            
        # If the word is not present in any of the two dictionaries,
        # then don't do anything. 
        # We ignore words that are not part of the vocabulary.

    # print('P(Spam|message):', p_spam_given_message)
    # print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        # print('Label: Ham')
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        # print('Label: Spam')
        return 'spam'
    else:
        # print('Equal proabilities, have a human classify this!')
        return 'needs human classification'

In [55]:
test_df['predicted'] = test_df['SMS'].apply(classify_test_set)
test_df.head()

,Label,SMS,predicted
2131,ham,Later i guess. I needa do mcat study too.,ham
3418,ham,But i haf enuff space got like 4 mb...,ham
3424,spam,Had your mobile 10 mths? Update to latest Oran...,ham
1538,ham,All sounds good. Fingers . Makes it difficult ...,ham
5393,ham,"All done, all handed in. Don't know if mega sh...",ham


Now we can compare the predicted values with the actual values to measure how good our spam filter is with classifying new messages. To make the measurement, we'll use accuracy as a metric:

$ \displaystyle \text{Accuracy} = \frac{\text{number of correctly classified messages}}{\text{total number of classified messages}} $

In [56]:
correct = 0
total = len(test_df) # number of messages in the test set

#iterate through each row of dataframe
for index, row in test_df.iterrows():
    if row['predicted'] == row['Label']:
        correct += 1
        
print('Correct:', correct)
print('Incorrect:', total - correct)
print('Accuracy:', round(correct/total*100, 2), '%')

Correct: 965
Incorrect: 149
Accuracy: 86.62 %


## 10. Next Steps